# 7.3 Least Squares Optimization and Pose Graph SLAM 

In the last two chapters, we have looked at two of the most fundamental problems in probabilistic mobile robotics: ***localization*** and ***mapping***. In the localization chapter we saw how, knowing the map of the environment that the robot is moving through, it is possible to use sensory readings to correct errors in the odometry and figure out the pose of the robot. In the mapping chapter, we learnt how to integrate the sensory measurements taken from multiple different poses to *build* a map, using the probabilistic nature of the observation model to account for the uncertainty in the measurements.

However, this is a bit of a chicken-and-egg problem: if we need to know the map before we can localize ourselves, but we need to know our pose to build the map, how can we ever do anything? The solution, it turns out, is to do both things at once: **Simultaneous Localization And Mapping (SLAM)**.

<figure  style="text-align:center;">
    <video width="720" controls>
    <source src="images/posegraph/slam.mp4" type="video/mp4">
    </video>
    <figcaption>If we try to use the sensor readings to build a map, but do not know the pose the reading was taken from, the resulting map is going to be quite bad.</figcaption>
</figure>

As you can imagine, this is a fairly complex problem in which the uncertainty can easily get out of hand, as we are constantly basing imperfect observations on top of imperfect pose estimations. Luckily for us, very smart people have come up with multiple approaches to tackle this problem, including variations of techniques we are already familiar with, such as [particle filters](https://openslam-org.github.io/gmapping.html) and the [EKF](http://ais.informatik.uni-freiburg.de/teaching/ws12/mapping/pdf/slam04-ekf-slam.pdf). In this chapter, however, we are going to focus on a new technique: the **Pose Graph**.

The reasoning behind most SLAM methods (including Pose Graphs) is as follows: in order to build an occupancy map from laser scanner readings, we need to know where each reading was taken from. Thus, if we can find the sequence of poses that the robot has moved through, we can then use those poses to project the laser pointclouds into world space, and build the map with the same method we saw in the *occupancy grid mapping* notebook. 

## 7.3.1 Pose Graphs

At its core, the idea of Pose Graphs is quite simple: we are going to represent the trajectory of the robot with a **graph**, where:
- each **node** represents a pose, and
- the **arcs** between the nodes (which are sometimes referred to as *factors*) represent the expected relative transformation between the poses, based on whatever sensory information we are able to use. 

For example, if we only have odometry, we might end up with something like this:

<br /><center><figure style="text-align:center; width:700px">
  <img src="images/posegraph/graph-odom.png" alt="">
  <figcaption>Node graph with odometry information only.</figcaption>
</figure></center><br />

Let's assume we want to define the starting pose as $pose_1 = [0, 0, 0] ^T$. This graph tells us that, according to our data, 
$\\[2pt]$ $$pose_2 = pose_1 \oplus u_1$$ and $$pose_3 =  pose_2 \oplus u_2$$ $\\[2pt]$ 
The idea, then, is to find the values for each of the poses that satisfy this system of equations. However, you can probably see that this is not going to be enough to solve our problem, since the solution to those equations is just the odometry-based trajectory. As we know, the odometry accumulates error over time (*drift*), and this formulation does not allow us to correct that error. 

### Loop closures

In order to make this helpful, then, we need to incorporate more than just the odometry. We need to use some other source of sensory information to create more arcs between the nodes -- or, put another way, *add more equations* to the system. This is called a **loop closure**: an observation that lets us relate a new pose to some pre-existing node in the graph (creating a loop):

<br><center><figure style="text-align:center; width:700px">
  <img src="images/posegraph/graph-loop-closure.png" alt="">
<figcaption>Node graph with a loop closure. The loop closure is based on finding a match between the observations taken from poses 1 and 3. This is produced, for example, by <b>a common observed landmark</b> or by <b>scan registration</b>.</figcaption> 
</figure></center><br>

Now, our system of equations looks like this: 

\begin{split}
pose_1 &= [0,0,0]^T \\
pose_2 &= pose_1 \oplus u_1 \\
pose_3 &= pose_2 \oplus u_2 \\
pose_1 &= pose_3 \oplus p_z^{31}
\end{split}

And now we have an interesting situation. If our odometry and sensor measurements are unreliable (which they are), we can in general expect that: $\\[2pt]$

$$
pose_1 \oplus u_1 \oplus u_2 \oplus p_z^{31} \neq pose_1
$$ 

In other words, the loop does not match! This, of course, is precisely what we want to correct.

In order to tackle this, let's consider our situation. Remember that, in the previous case (pure odometry), we had a system of equations with three equations, and three poses to estimate. Now, we have four equations, but still three poses: we have an **overdetermined system**. 

## 7.3.2 Overdetermined Systems and Least Squares

In general, when dealing with a problem that involves an overdetermined system, it is not possible to find a perfect solution, as equations that are derived from observations are subject to sensor noise. Let's think about the simplest possible example, where we have a single variable. Let's say that we want to figure out the distance from the robot to a wall (let's call this distance $d$), and we have a range sensor that will provide us with a distance measurement. Due to sensor noise, we might find that after taking three measurements, our equations look like this:

\begin{split}
d &= 1.00 \\ 
d &= 0.95 \\
d &= 1.05 \\
\end{split}

Evidently, there is no value of $d$ that will satisfy all three equations. However, it seems intuitive that there is a solution $\hat d$ which *best matches* the set of observations, by making the **sum of all the errors** be as small as possible. In practice, it is better to minimize the **sum of the *squared* errors** instead, as this has multiple advantages (which are beyond the scope of this discussion). For example, in this case, if $\hat d=1.0$ then 

  * $e_1=(\hat d - d_1)^2=0$,

  * $e_2=(\hat d - d_2)^2=0.0025$ and 

  * $e_3=(\hat d - d_3)^2=0.0025$, 

so $e_1+e_2+e_3 = 0.005$, 

and there is no other value of $\hat d$ that provides a lower total squared error. It makes sense, then, to consider $\hat d=1.0$ the best guess for the correct value of $d$, even though some error still exists.

This is a **very** widely used technique, called **Least Squares Optimization**. Let's explore this technique a little bit, looking at some simple examples, before we return to the SLAM problem.

### Linear Least Squares

A linear system of equations can be represented with a matrix $H$, which contains the coefficients of each of the variables in the equations. If we are trying to estimate a vector $x$ based on observations $z$, this would be expressed as 

$$
z = H x
$$

Here, $H$ codifies our observation model. For example:

\begin{equation*}
\left\{
\begin{split}
c &= a \cdot x_1 + b \cdot x_2  \\
f &= d \cdot x_1 + e \cdot x_2  \\
\end{split} \right.
\end{equation*}

\begin{equation*}
\begin{bmatrix} c \\ f \end{bmatrix}
=
\begin{bmatrix} 
    a & b \\ d & e 
\end{bmatrix} \cdot \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
\end{equation*}

Generally, we want the same number of equations ($n$) as variables ($m$). Remember that each observation $z$ defines an equation. In such a case, where $n=m$, a simple solution to this problem exists:
$$x = H ^{-1} z$$

So a unique solution exists if $H$ is invertible; that is, if $H$ is a square matrix with $det(H) \neq 0$.

However, when dealing with an overdetermined system, $H$ is not a square matrix, and thus not invertible. In such cases, we can obtain the **least squares** solution by using the *pseudo-inverse* of the matrix $H$:

$$ \hat x = \arg\min_x e^T e  = \arg\min_x [(z-Hx)^T(z-Hx)] = \arg\min_x || z-Hx ||^2$$

$$\hat{x} = \underbrace{(H^T H)^{-1} H^T}_{\textit{pseudo-inverse }(H^+)} z$$

Curious about how is that expression reached?

To find the estimate $\hat{x}$ that minimizes $|| z-Hx ||^2$, we take its derivative with respect to $x$ and set it to zero:

$$
\frac{\partial || z-Hx ||^2}{\partial x} = -2H^T(z-Hx) = 0
$$

Rearranging this equation we get:

$$
H^THx = H^T z
$$

And finally, assuming $H^TH$ invertible, we can solve for $\hat{x}$ as:

$$\hat{x} = (H^T H)^{-1} H^T z$$

This is the **Least Squares solution** for the state vector $x$.

In [ ]:
import numpy as np
from scipy import optimize, stats, spatial
from IPython.display import display, clear_output
import ipywidgets as widgets
import time
import functools
import matplotlib.pyplot as plt

import sys
sys.path.append("..")
from utils.DrawRobot import DrawRobot
from utils.tcomp import tcomp, inv_tcomp
from utils.laser.laser2D import Laser2D
from utils.unit7.occupancy_mapping import OccupancyGridMap, process_new_observation, plot_grid_map
from laserUtils import LaserUtils

### **<span style="color:green"><b><i>ASSIGNMENT 1: Playing with a robot in a corridor</i></b></span>** 

The following code illustrates a simple scenario where a robot is in a corridor looking at a door, which is placed at the origin of the reference system (see Fig.1). The robot is equipped with a laser scanner able to measure distances, and takes a number of observations $z$. The robot is placed 3 meters away from the door, but this information is unknown for it. **Your goal is** to estimate the position of the robot in this 1D world using those measurements. $\\[5pt]$

<figure style="text-align:center">
  <img src="images/corridor.png" alt="">
  <figcaption>Fig. 1: Simple 1D scenario with a robot equipped with a laser scanner measuring distance to a door.</figcaption>
</figure>

The following code cell shows the dimensions of all the actors involved in LS-positioning. Complete it for computing the robot pose $x$ from the available information. *Recall [`np.linalg.inv()`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html) to obtain the inverse of a matrix.* 

*Note that with this configuration the laser is directly measuring the robot position. If the laser was measuring distances an object placed at a different location $m_i$, then the equation should be $z_i=Hx-m_i$.*

In [ ]:
# Set the robot pose to unknown
x = np.vstack(np.array([None]))

# Sensor measurements to the door
z = np.vstack(np.array([3.7,2.9,3.6,2.5,3.5]))

# Observation model
H = np.ones(np.array([5,1]))

print ("Dimensions:")
print ("Pose x:         " + str(x.shape))
print ("Observations z: " + str(z.shape))
print ("Obs. model H:   " + str(H.shape))
print ("H.T@H:          " + str((H.T@H).shape))
print ("inv(H.T@H):     " + str((np.linalg.inv(H.T@H)).shape))
print ("H.T@z :         " + str((H.T@z).shape))

# Do Least Squares Positioning
x = None

print('\nLS-Positioning')
print('x = ' + str(x[0]))

<span style="color:blue">Expected output</span>

```
x = [3.24]
```

## 7.3.3 Weighted measurements

In cases where multiple sensors affected by different noise profiles are used, or in those where the robot is using a sensor with a varying error (*e.g.* typically radial laser scans are more accurate while measuring distances to close objects), it is interesting to weight the contribution of such measurements while retrieving the robot pose. For example, we are going to consider a sensor whose accuracy drops the further the observed landmark is. Given a *covariance* matrix $Q$ describing the error in the measurements, the equations above are rewritten as:

  $$ \hat x = \arg\min_{x} e^T Q^{-1} e = [(Hx-z)^TQ^{-1}(Hx-z)] $$

  $$
    \begin{aligned}
      &\hat{x} \leftarrow (H^T Q^{-1} H)^{-1} H^T Q^{-1} z &\text{(1. Best estimation)}\\ 
      &\Sigma_{\hat{x}} \leftarrow (H^T Q^{-1} H)^{-1} &\text{(2. Uncertainty of the estimation)}\\
    \end{aligned}
  $$
  
  Example with three measurements having different uncertainty ($\sigma_1^2$, $\sigma_2^2$, $\sigma_3^2$):
  
  $$
  e^T Q^{-1} e = [e_1 \; e_2 \; e_3]
  \begin{bmatrix} 1 / \sigma_1^2 & 0 & 0 \\ 0 & 1/\sigma_2^2 & 0 \\ 0 & 0 & 1/\sigma_3^2 \end{bmatrix} 
  \begin{bmatrix} e_1 \\ e_2 \\ e_3 \end{bmatrix}
  = \frac{e_1^2}{\sigma_1^2} + \frac{e_2^2}{\sigma_2^2} + \frac{e_3^2}{\sigma_3^2}
  = \sum_{i=1}^m \frac{e_i^2}{\sigma_i^2}
  $$
  
  In this way, the bigger the $\sigma_i^2$, the smaller its contribution to the pose's computation.
  
  

### **<span style="color:green"><b><i>ASSIGNMENT 2: Adding growing uncertainty</i></b></span>** 

We have new information! The manufacturer of the laser scanner mounted on the robot sent us an email, telling us that the device is considerably less accurate for large distances. Concretely, such uncertainty is characterized by $\sigma^2=e^z$ (the laser is not great, being polite).

With this new information, implement the computation of the weighted LS-positioning so you can compare the previously estimated position with the new one.

In [ ]:
# Sensor measurements to the door
z = np.vstack(np.array([3.7,2.9,3.6,2.5,3.5]))

# Uncertainty of the measurements
Q = np.eye(5)*np.exp(z)

# Observation model
H = np.ones(np.array([5,1]))

# Do Least Squares Positioning
x = None

# Do Weighted Least Squares Positioning
x_w = None

print('\nLS-Positioning')
print('x = ' + str(x[0]))

print('\nWeighted-LS-Positioning')
print('x = ' + str(np.round(x_w[0],2)))

<span style="color:blue">Expected output</span>

```
LS-Positioning
x = [3.24]

Weighted-LS-Positioning
x = [3.01]
```

## 7.3.4 Non-linear Least Squares

So far, we have seen how to find the least squares solution for linear systems of equations, which can be represented with a matrix $H$. However, our pose graph is built from pose composition, which is a non-linear operation.

Sadly, there is no closed-form solution for non-linear least squares. Still, we can try to find a solution through iterative optimization. There are many optimization methods that will take in an initial guess for the solution, and keep updating that guess until they find a minimum in the error function:

$$ \hat x = \arg\min_x || z-h(x) ||^2$$

Despite the fact that many approaches exist, they all follow the same structure:

```python
# generic non-linear solver
def optimize(initial_guess, convergence_threshold):
    guess = initial_guess
    while(error_function(guess) > convergence_threshold):
        # most methods use the first and second derivatives of the error function
        # to calculate the best increment
        increment = calculate_best_increment(guess) 
        guess += increment
    return guess
```

This means that, so long as we can tell the solver how to compute the `error(guess)` and (for some methods) its derivatives, the solver can deal with any optimization problem! This abstraction allows us to focus on defining *the problem*, which we know and understand, rather than the numerical methods to find the solution. Neat!

However, there are also some bad news. In general, there is no guarantee that the solution $\hat x$ that these methods produce is the global minimum. Iterative methods are always vulnerable to converging on local minima when applied to non-convex functions, although there [some techniques](https://en.wikipedia.org/wiki/Global_optimization) (which we will not worry about today) to *try* and escape them.

### A brief pause to collect our thoughts

Ok, so to sum up what we have talked about so far:

- A **pose graph** defines the relative transforms that we expect between the poses in our trajectory. These relative poses will initially be derived from the odometry. We can also express the graph as a system of equations, using these relative transformations as the equations, and the poses as the unknowns.
- **Loop closures** in the graph allow us to include additional constraints to our system of equations, which will make our system overdetermined and allow us to correct the odometry errors.
- **Least squares** is the standard method to find a solution for overdetermined systems. It will not perfectly match all the equations (as that is not possible), but will minimize the error. 
- Linear least squares can be solved directly in matrix form, but non-linear problems require iterative methods (like Gauss-Newton).
- Non-linear LS solvers are problem-agnostic algorithms: you do not have to implement them from scratch. You only need to tell the solver how to compute the error for a given solution and (possibly) its derivatives.

Ok, then. It seems what we have to do is define our problem (pose graph optimization) in such a way that we can **calculate the error of a given solution**, and then use a **non-linear least-squares solver** to minimize that error. Let's get on that. 

### <font color="blue"><b><i>Thinking about it (1)</i></b></font>

At this point you are able to **address the following points**:

- We talk in the previous sections about *linear* and *non-linear* systems. What do we mean by this? What determines whether we are dealing with a linear or non-linear problem?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>  

- How is this *error minimization* relevant to the problem of SLAM? What are the errors that we are going to minimize?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>  

- What makes the problem of pose graph optimization non-linear? 

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>  
    

## 7.3.5 Defining the pose graph

### **<span style="color:green"><b><i>ASSIGNMENT 3: Nodes and Factors</i></b></span>** 

Let's start simple. You are provided with a class `Node` which represents each of the poses that go into the graph. On top of the pose, nodes also have a boolean field `is_fixed`, which controls whether the solver is allowed to move them. This is mainly there because we want to make sure the starting pose remains unchanged.

In [ ]:
class Node:
    def __init__(self, pose : np.ndarray, fixed: bool = False):
        self.pose = np.array(pose)
        self.is_fixed = fixed

You are tasked with implementing a class `Factor`, which represents the relation between two nodes (an arc in the graph). Remember that each of these arcs codifies the expected relative transformation between the two poses.

This class will store the following information:

- The two nodes ($node_1$ and $node_2$) in the graph which it links together. These will just be stored as indices into a list, which is kept elsewhere.

- The expected transformation between the pose of $node_1$ ($pose_1$) and the pose of $node_2$ ($pose_2$), which will be provided by odometry or loop closure detection (a common observed landmark or scan registrantion providing a relative transformation). It's important to note that this transformation is expressed in the system of reference of $pose_1$:
$$
expected = pose_1 \ominus pose_2
$$

- The weight associated with this factor. As we saw in the linear case before, this weight is just the inverse of the covariance matrix of the relative pose: $W = Q^{-1}$. 


The `Factor` class also provides a method, `evaluate_error()`, which computes how well a solution (specific values for $pose_1$ and $pose_2$) matches its expected transformation.

$$
error = e^TQ^{-1}e,
$$
with

$$
e = pose_2 - (pose_1 \oplus expected)
$$

In [ ]:
class Factor:
    def __init__(self, node1_ind, node2_ind, relative_pose, relative_pose_covariance):
        self.node1_ind = node1_ind
        self.node2_ind = node2_ind
        self.expected_transform = relative_pose
        self.weight = None
    
    def evaluate_error(self, pose1, pose2):
        expected_p2 = None
        difference = None
        
        # always take the shortest difference between the angles
        ang = difference[2] % (2 * np.pi)
        ang = ang if ang<np.pi else -(2*np.pi -ang) 
        difference[2] = ang

        v = None
        return v

### **<span style="color:green"><b><i>ASSIGNMENT 4: Connecting the nodes into a graph</i></b></span>** 

Now that we have both the nodes and the factors that connect them to each other, we can build a graph. The provided class `PoseGraph` will be used to hold all the data related to the nodes and factors.

This class provides some methods to handle visualization, which you don't need to worry about. It also handles a bit of inconvenient translation work: least-squares solvers expect the parameters (the elements of the solution) to be a single, 1D list of numbers, but we have a list of *poses* that we want the solver to modify. So, the methods `solver_params_from_nodes()` and `set_nodes_from_solution()` are provided to flatten and un-flatten the poses as required.

In [ ]:
class PoseGraph:
    def __init__(self):
        self.nodes : list[Node] = []
        self.factors : list[Factor] = []
        self.markers = [] # for visualization

    # get a single array with all the flattened poses concatenated (for the optimizer)
    def solver_params_from_nodes(self):
        params = np.zeros(3 * len(self.nodes))
        
        # the "bounds" arrays are used to prevent  the solver from touching fixed nodes
        upper_bound = np.full(params.shape, np.inf)
        lower_bound = np.full(params.shape, -np.inf)
        
        i = 0
        for node in self.nodes:
            params[i] = node.pose[0,0]
            params[i+1] = node.pose[1,0]
            params[i+2] = node.pose[2,0]

            if(node.is_fixed):
                upper_bound[i] = node.pose[0,0]
                upper_bound[i+1] = node.pose[1,0]
                upper_bound[i+2] = node.pose[2,0]

                lower_bound[i] = upper_bound[i] 
                lower_bound[i+1] = upper_bound[i+1]
                lower_bound[i+2] = upper_bound[i+2]

            i+=3

        bounds = optimize.Bounds(ub=upper_bound, lb=lower_bound, keep_feasible=True)
        return params, bounds
    
    def set_nodes_from_solution(self, solution):
        i = 0
        for node in self.nodes:
            node.pose[0,0] = solution[i]
            node.pose[1,0] = solution[i+1]
            node.pose[2,0] = solution[i+2]
            i+=3

    def set_output(self, fig, ax):
        if not hasattr(self, 'intermediate_stdout'):
            self.intermediate_stdout = widgets.Output()
        self.fig = fig
        self.ax = ax
        
    def visualize(self):
        self.__remove_markers()
        for node in self.nodes:
            marker = DrawRobot(self.fig, self.ax, node.pose, color="green")
            self.markers.extend(marker)
        self.show()

    def draw_intermediates(self, intermediate_result):
        # print some numbers. Some solvers just return the current solution without any stats, so we might have to skip this
        if type(intermediate_result) is optimize.OptimizeResult:
            with self.intermediate_stdout:
                print(f"\rResidual: {intermediate_result.fun}                                      ", end='')
        
        # plot the node poses in the map
        flattened_poses = intermediate_result.x if type(intermediate_result) is optimize.OptimizeResult else intermediate_result
        i= 0
        while i < len(flattened_poses):
            pose = flattened_poses[i:i+3]
            marker = DrawRobot(self.fig, self.ax, np.vstack(pose), color="green")
            self.markers.extend(marker)
            i+=3

        # show to the screen and remove the markers from the figure for next iteration
        self.show()
        self.__remove_markers()

    def __remove_markers(self):
        for marker in self.markers:
            marker.remove()
        self.markers = []

    def show(self):
        if not hasattr(self, 'display_handle'):
            self.display_handle = display(self.fig, display_id=True)
            self.intermediate_stdout_handle = display(self.intermediate_stdout, display_id=True)
        else:
            self.display_handle.update(self.fig)
            self.intermediate_stdout_handle.update(self.intermediate_stdout)

Let's populate this graph! **You will need to implement two functions**:

- `add_odom_factor()` will receive the estimated current robot pose (pure odometry), and the relative transformation that was used to compute it from the previous pose, along with the covariance of this relative transform. This function must create a new node to represent this new pose and add it to the graph. Then, it must connect it to the previous most-recent node through a factor which represents this odometry-based relative transformation.

- `add_loop_closure()` receives two arbitrary pose indices (which refer to the `graph.nodes` list), and creates a new factor that connects them. The arguments of the function also include the relative transform and its uncertainty which are associated to this factor.

In this notebook, we are not going to worry about how the loop closures are detected. Depending on the sensors that the robot has access to, this process could be done by **re-observing landmarks** (visual sensors) or by running point-cloud registration algorithms like **ICP** (laser scanners). The key takeaway is that, once the loop closure has been detected, the solver does not care about how that detection happened.

In [ ]:
def add_odom_factor(graph: PoseGraph, new_pose, odometry_transform, odom_covariance):
    node_count = len(graph.nodes)
    most_recent_node_ind = node_count-1
    graph.nodes.append(None)
    graph.factors.append(None)

def add_loop_closure(graph: PoseGraph, pose1_ind, pose2_ind, relative_transform, loop_covariance):
    graph.factors.append(None)

You can use the following code to make sure the factors have been defined corretly:

In [ ]:
def print_graph_data(graph: PoseGraph):
    for factor in graph.factors:
        print(f"Pose {factor.node1_ind} -> Pose {factor.node2_ind}:\n{factor.expected_transform}")

def test_graph():
    graph=PoseGraph()
    pose = np.vstack([0,0,0])
    graph.nodes.append(Node(pose, fixed=True))
    
    movement = np.vstack([0.1, 0., 0.2])
    movement_cov = np.diag([0.1, 0.1, 0.1])
    for i in range(4):
        add_odom_factor(graph, pose, movement, movement_cov)

    add_loop_closure(graph, 
                     pose1_ind=4, pose2_ind=2, 
                     relative_transform=np.vstack([3,3,0]), 
                     loop_covariance=np.diag([0.01, 0.01, 0.01]))
    print_graph_data(graph)

test_graph()

Expected output:
```
Pose 0 -> Pose 1:
[[0.1]
 [0. ]
 [0.2]]
Pose 1 -> Pose 2:
[[0.1]
 [0. ]
 [0.2]]
Pose 2 -> Pose 3:
[[0.1]
 [0. ]
 [0.2]]
Pose 3 -> Pose 4:
[[0.1]
 [0. ]
 [0.2]]
Pose 4 -> Pose 2:
[[3]
 [3]
 [0]]
```

### Getting the total error and calling the solver 

With our `PoseGraph` defined, we can now run the solver to find the optimal sequence of poses that "satisfies" both the odometry and the loop closures. The provided functions `evaluate_total_error()` and `run_solver()` do just that: define the error function that the solver should minimize (the sum of the weighted squared errors)

$$
e = \sum_i e_i^T Q_i^{-1} e_i
$$

, and configure the solver to run the optimization process with this error function.

In [ ]:
def evaluate_total_error(graph: PoseGraph, flattened_poses):
    total = 0.
    for factor in graph.factors:
        pose1 = np.vstack(flattened_poses[factor.node1_ind*3 : factor.node1_ind*3+3])
        pose2 = np.vstack(flattened_poses[factor.node2_ind*3 : factor.node2_ind*3+3])
        total += None
    return total

In [ ]:

def run_solver(graph: PoseGraph, method = 'Powell'):
    parameters, bounds = graph.solver_params_from_nodes()
    error_func = functools.partial(evaluate_total_error, graph)

    # Run the solver!
    result : optimize.OptimizeResult = optimize.minimize(
            error_func, # error function (what we are trying to minimize!) This is not a *value*, but a *callable function*. The solver will repeatedly call it to evaluate many possible solutions
            x0 = parameters, # initial guess
            bounds=bounds, # this is used to specify limits to the value of the parameters. In our case, to keep the starting pose static
            method=method, 
            tol=1e-8, 
            options={
                "maxiter" : 10000,
                "ftol" : 1e-2
            },
            callback=graph.draw_intermediates
            )
    print(f"Finished optimization with status : {result.status} after {result.nit} iterations. \nResidual: {result.fun}\nMessage: '{result.message}'")
    
    # apply the solution we found to our list of nodes
    graph.set_nodes_from_solution(result.x)

## 7.3.6 Using the pose graph

Let's now use this whole structure we have built! We are going to use a `Robot` class to move around the environment. Every time we move, we will add an odometry node to the pose graph. Then, at arbitrary points of the exploration, we will simulate a loop closure (imagine we have run ICP and found a match between the point clouds of the sensor).

Once we have a substantial graph, we'll run the optimization, and see if the loop closures are enough to correct the odometry errors.

The robot is going to keep track of where it has been through a list of `KeyPose`. Each of these poses corresponds to one of the nodes in the graph, but also stores the readings of a laser scanner, which will be used to build the map once we have optimized the pose graph.

In [ ]:
class KeyPose:
    def __init__(self, true_p, estimated_p, z):
        self.true_pose = true_p
        self.odom_pose = estimated_p
        self.estimated_pose = estimated_p
        self.z = z

### **<span style="color:green"><b><i>ASSIGNMENT 5: Exploring the environment</i></b></span>** 

Complete the `Robot` class below. Specifically, the methods `move_and_observe()` and `simulate_sensor_correspondence()`, which will add nodes and factors to the `PoseGraph` that we defined earlier.

Note that, unlike in previous notebooks, the `move_and_observe()` method should apply the **correct** movement to the robot's true pose, and add a random error to the odometry. This simulates teleoperation: during a SLAM session, the user would be controllling the movement of the robot directly, and making sure it follows a specific path. However, the internal firmware of the robot, which handles the odometry, does not know what that path is, and can only reconstruct the movements from the imperfect odometry.

In [ ]:
sim_map = LaserUtils.create_room_map()

class Robot:
    def __init__(self, intial_pose):
        self.true_pose : np.ndarray = intial_pose
        self.estimated_pose : np.ndarray = intial_pose

        self.factor_graph = PoseGraph()
        self.factor_graph.nodes.append(Node(self.estimated_pose, fixed=True))

        self.laser = Laser2D(np.deg2rad(240), 0.05, 20, np.diag([0.001,0.001]), np.vstack([0,0,0]))
        self.history_poses : list[KeyPose] = []
        self.take_observation()

    def move_and_observe(self, movement, odom_cov):
        odom = None
        self.true_pose = None
        self.estimated_pose = None

        add_odom_factor(self.factor_graph, None, None, None)
        self.take_observation()
    
    def take_observation(self):
        self.laser.set_pose(self.true_pose)
        z = self.laser.take_observation(sim_map)
        self.history_poses.append(KeyPose(self.true_pose, self.estimated_pose, z))
        
    def simulate_sensor_correspondence(self, measurement_covariance, pose1_index=None, pose2_index=None):
        # if optional indices are missing, just assume we are referring to the most recent poses
        if pose1_index is None: # don't change this!
            pose1_index = len(self.history_poses)-1
        if pose2_index is None: # don't change this!
            pose2_index = len(self.history_poses)-2

        # find the relative transformation between the specified poses
        ideal_transformation = inv_tcomp(self.history_poses[pose1_index].true_pose,
                          self.history_poses[pose2_index].true_pose)
        
        noisy_transformation = None
        
        # create a new factor in the graph between the nodes
        add_loop_closure(self.factor_graph, None, None, None, None)

    def visualize(self, fig, ax):
        DrawRobot(fig, ax, self.true_pose, color="red")
        DrawRobot(fig, ax, self.estimated_pose, color="blue")
        

### Running it!

Ok, time to see the pose graph optimization in action! Run the code in the cell below to see the path followed by the robot and the odometry estimation. Once the robot stops moving, the solver will try to improve on this odometry estimation by using the loop closures that we simulate along the way.

In [ ]:
def run_slam_optimization(robot: Robot):
    # Solver methods: L-BFGS-B, Powell, SLSQP, COBYQA, trust-constr
    run_solver(robot.factor_graph, method='Powell')
    
    # copy the results of the optimization to the robot's internal list of poses
    for i in range(len(robot.factor_graph.nodes)):
        robot.history_poses[i].estimated_pose = robot.factor_graph.nodes[i].pose


# Dummy class used to indicate that we want a loop closure to happen 
class LoopClosure:
    pass

fig, ax = LaserUtils.plot_virtual_map(sim_map);
clear_output(wait=False)

robot = Robot(np.vstack([4, 4, 0]))
robot.factor_graph.set_output(fig, ax)
robot.visualize(fig,ax)
movement_cov = np.diag([0.1, 0.02, 0.02])

moves= [
    np.vstack([1.0, 0., 0.]),
    np.vstack([1.0, 0., 0.]),
    np.vstack([1.0, 0., np.pi / 4]),
    np.vstack([1.0, 0., np.pi / 4]),
    LoopClosure(),

    np.vstack([1.0, 0., 0.]),
    np.vstack([1.0, 0., 0.]),
    np.vstack([1.0, 0., np.pi / 4]),
    np.vstack([1.0, 0., np.pi / 4]),
    LoopClosure(),

    np.vstack([1.2, 0., 0.]),
    np.vstack([1.2, 0., 0.]),
    np.vstack([1.0, 0., np.pi / 4]),
    np.vstack([1.0, 0., np.pi / 4]),
    LoopClosure(),

    np.vstack([1.0, 0., 0.]),
    np.vstack([1.0, 0., 0.]),
    np.vstack([1.0, 0., np.pi / 4]),
    np.vstack([1.0, 0., np.pi / 4]),
    LoopClosure()
]

np.random.seed(seed=233425)
for i in range(len(moves)):
    move = moves[i]
    if isinstance(move, LoopClosure):
        measurement_covariance = np.diag([0.005, 0.005, 0.002])
        robot.simulate_sensor_correspondence(measurement_covariance, pose2_index=0)
    else:
        robot.move_and_observe(move, movement_cov)
        robot.visualize(fig,ax)
        robot.factor_graph.show()
        time.sleep(0.2)

run_slam_optimization(robot)

plt.close()

Expected output:
<figure style="text-align:center; width:700px">
  <img src="images/posegraph/optimized_graph.png" alt="">
</figure>

Nice! With this pose graph optimization, we have managed to significantly improve the localization of the robot. Now that we know where the robot has been, we can use its sensory readings to build the map. Borrowing some code from the previous notebooks, we could do something like this:

In [ ]:
def build_map_odometry():
    map_width = np.nanmax(sim_map[0, :])
    map_height = np.nanmax(sim_map[1, :])
    occupancy_map = OccupancyGridMap(map_width, map_height, 0.1)

    for pose in robot.history_poses:
        process_new_observation(occupancy_map, pose.odom_pose, pose.z)
    plot_grid_map(occupancy_map)

def build_map_corrected():
    map_width = np.nanmax(sim_map[0, :])
    map_height = np.nanmax(sim_map[1, :])
    occupancy_map = OccupancyGridMap(map_width, map_height, 0.1)

    for pose in robot.history_poses:
        process_new_observation(occupancy_map, pose.estimated_pose, pose.z)
    plot_grid_map(occupancy_map)

print("Using only odometry:")
build_map_odometry()
print("With optimized pose graph:")
build_map_corrected()

As we can see, the pose graph allows us to get a quite good map, while blindly trusting odometry results on a very bad one. While the "good" map is still not perfect, we know that, in general, taking more observations would allow us to refine its imperfections by integrating their information in our probabilistic filter.

### <font color="blue"><b><i>Thinking about it (2)</i></b></font>

Before we finish, let's **consider the following questions**:

- Can you explain, in simple terms, how defining a pose graph helps us solve the SLAM problem?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>  

- In this notebook, we only ran the mapping algorithm at the end of the process. That doesn't seem very *simultaneous*, does it. Can you think of how the algorithm might work if we needed to always have the most up-to-date map accessible to the robot?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>  

- What happens if we change the number (or frequency) of loop closures? What if we change the covariance of the relative transformation that those loop closures find between the poses?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>  
    